In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.getcwd())
import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import ipdb

tf.reset_default_graph()
dim_z = 120

DATA_DIR = '/home/qxy/Desktop/datasets/Car/data_annotation/train_list.json'
EMBEDDINGS_FILE = '/home/qxy/Desktop/beta/weight/pretrained_weight/embeddings/BigBiGAN_Car_q_z.npy'
img_dir = '/home/qxy/Desktop/datasets/Car/data'
# Load BigBiGAN module.
paths = '/home/qxy/Desktop/BigGan/weight_result/weights/biggan_resnet50_1'
# ipdb.set_trace()
module = hub.Module(paths)  # small encoder
# module = hub.Module('https://tfhub.dev/deepmind/bigbigan-revnet50x4/1')  # large encoder

z = tf.random.truncated_normal([24, 120])  # latent samples
gen_samples = module(z, signature='generate')

images = tf.placeholder(tf.float32, shape=[None, 256, 256, 3])
features = module(images, signature='encode', as_dict=True)
z_sample = features['z_sample']  # shape [?, 120]

#通过z_sample生成图像 or 通过图像生成z_sample
recons = module(z_sample, signature='generate')  # shape [?, 128, 128, 3]


sess = tf.compat.v1.InteractiveSession()
initializer = tf.compat.v1.global_variables_initializer()
sess.run(initializer)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2022-05-22 22:18:03.070126: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-22 22:18:03.070478: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: NVIDIA GeForce RTX 2060 major: 7 minor: 5 memoryClockRate(GHz): 1.68
pciBusID: 0000:01:00.0
2022-05-22 22:18:03.070613: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory
2022-05-22 22:18:03.070660: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcublas.so.10.0'; dlerror: libcublas.so.10.0: cannot open shared object file: No such file or directory
2022-05-22 22:18:03.070701: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not lo

In [ ]:
from torchvision import transforms
from data import FiledDataset

ds = FiledDataset(
    img_dir,
    DATA_DIR,
    transform=transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        lambda x: (2.0 * x - 1.0).permute(1, 2, 0).numpy()
    ]))


gen_np = np.stack([ds[i] for i in range(10, 20)])
recons_np = sess.run(recons, feed_dict={images: gen_np})

plt.figure(figsize=(10, 2), dpi=200)
batch = gen_np.shape[0]

for i in range(batch):
    plt.subplot(2, len(gen_np), i + 1)
    plt.axis('off')
    plt.imshow(0.5 * gen_np[i] + 0.5)

    plt.subplot(2, len(gen_np), len(gen_np) + i + 1)
    plt.axis('off')
    plt.imshow(0.5 * recons_np[i] + 0.5)

In [6]:
from tqdm import tqdm

batch_size = 100
emdeddings = []

all_indices = np.arange(0, len(ds))
count = len(all_indices)
for batch_start in tqdm(np.arange(0, count, batch_size), total=count // batch_size):
    low, up = batch_start, min(batch_start + batch_size, count)
    gen_np = np.stack([ds[i] for i in all_indices[low: up]])
    emdeddings.append(sess.run(features, feed_dict={images: gen_np}))#这个run是在运行模型

 28%|██▊       | 23/81 [02:04<05:05,  5.27s/it]

In [7]:
zs = [e['z_sample'] for e in emdeddings]
zs = np.vstack(zs)
np.save(EMBEDDINGS_FILE, zs)

In [5]:
paths = '/home/qxy/Desktop/beta/weight/pretrained_weight/embeddings/BigBiGAN_Car_q_z.npy'
a = np.load(paths)
print(a.shape)

(60, 120)
